# Problem: Implement a CNN for CIFAR-10 in PyTorch

### Problem Statement
You are tasked with implementing a **Convolutional Neural Network (CNN)** for image classification on the **CIFAR-10** dataset using PyTorch. The model should contain convolutional layers for feature extraction, pooling layers for downsampling, and fully connected layers for classification. Your goal is to complete the CNN model by defining the necessary layers and implementing the forward pass.

### Requirements
1. **Define the CNN Model**:
   - Add **convolutional layers** for feature extraction.
   - Add **pooling layers** to reduce the spatial dimensions.
   - Add **fully connected layers** to output class predictions.
   - The model should be capable of processing input images of size `(32x32x3)` as in the CIFAR-10 dataset.

### Constraints
- The CNN should be designed with multiple convolutional and pooling layers followed by fully connected layers.
- Ensure the model is compatible with the CIFAR-10 dataset, which contains 10 classes.


<details>
  <summary>💡 Hint</summary>
  Add the convolutional (conv1, conv2), pooling (pool), and fully connected layers (fc1, fc2) in CNNModel.__init__.
  <br>
  Implement the forward pass to process inputs through these layers.
</details>

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

In [2]:
# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

100.0%


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [12]:
data = iter(train_loader)
x = next(data)
(img, cls) = x# Batch, channels, Height, Width(Default config for Conv2d)
conv_test = nn.Conv2d(in_channels=3, out_channels=8,kernel_size=3, padding=1)
conv_test(img).shape

# 그대로 먹여주면 됩니다. 알잘딱 해줄겁니다. 우리가 할건 계산밖에 없어요

torch.Size([64, 8, 32, 32])

In [6]:
# Define the CNN Model
# TODO: Add convolutional, pooling, and fully connected layers
# 스터디 Goals
# 1. 이미지에 대해 convolutional을 Conv2d를 활용해 적용하는 방법을 안다.
# 여기서 Conv2d의 파라미터의 역할과, 어떻게 차원이 변경하는지를 정리해준다.
# 2. 최종적으로 flatten을 진행해 FCN에 먹여야 한다. 이를 어떻게 차원을 찾고, 제대로 flatten 하는 방법
# 3. 아마 dtype 쪽 문제도 있을 가능성이 있다. 이 부분을 확인해야 하겠다.
# 레퍼런스는 토치쪽 docs다. 한번 F.ReLU, nn.ReLU도 살짝 다뤄야 할듯
# https://pytorch.org/docs/stable/generated/torch.nn.Conv2d.html, https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html, https://discuss.pytorch.org/t/whats-the-difference-between-nn-relu-vs-f-relu/27599
# 4. ReLU의 활용과 pooling을 어떨때 해야 하는지에 대한 정보가 필요하다
# https://cs231n.github.io/convolutional-networks/
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=8, kernel_size=3, stride=1, padding=1) # 데이터 shape은 64, 8, 32, 32입니다
        self.conv2 = nn.Conv2d(in_channels=8, out_channels=16, kernel_size=9, stride=1, padding=0) # 꽉 잡고 채널 수는 증가, 이미지 차원수는 감소하겠습니다. 64, 16, 24, 24로 바꿉니다.
        self.pool = nn.MaxPool2d(2, 2)
        self.fcn = nn.Sequential(
            nn.Linear(in_features=16*12*12, out_features=12*12),
            nn.ReLU(),
            nn.Linear(in_features=12*12, out_features=10)
        )
        self.relu = nn.ReLU()
    def forward(self, x):
        # cs231n을 확인해보니, 각 합성곱 레이어 다음에는 ReLU(활성화 함수)를 적용하고 최종적으로 차원을 확 줄여야 할때 pooling을 쓴다 한다.
        # 따라서 Conv와 ReLU 두번 하고 그다음에 pooling, 그 뒤에 FFN 하겠습니다.
        x = nn.functional.relu(self.conv1(x)) # 64, 8, 32, 32
        x = nn.functional.relu(self.conv2(x)) # 64, 16, 24, 24
        x = self.pool(x) # 64, 16, 12, 12
        x = torch.flatten(x, 1) # Batch까지 1차원으로 변경하면 큰일남
        x = self.fcn(x)

        return x

In [7]:
# Initialize the model, loss function, and optimizer
model = CNNModel()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
epochs = 10
for epoch in range(epochs):
    for images, labels in train_loader:
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

Epoch [1/10], Loss: 1.1098
Epoch [2/10], Loss: 1.0286
Epoch [3/10], Loss: 0.9075
Epoch [4/10], Loss: 1.0550
Epoch [5/10], Loss: 0.8824
Epoch [6/10], Loss: 1.0673
Epoch [7/10], Loss: 0.6515
Epoch [8/10], Loss: 1.1646
Epoch [9/10], Loss: 0.8996
Epoch [10/10], Loss: 1.1356


In [9]:
# Evaluate on the test set
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 62.54%
